In [1]:
%load_ext autoreload
%autoreload 2
from ctypes import util
import sys
import copy
from decimal import Decimal
import numpy as np

from numpy import sort
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.utils.misc import greedy_allocation_algorithm
from sturdy.utils.lazy import pick_one_allocation_algorithm, sorted_greedy_allocation_algorithm, equal_greedy_allocation_algorithm
from sturdy.validator.reward import calculate_aggregate_apy
from validator.simulator import Simulator
from sturdy.utils.misc import supply_rate

def calculate_apy(simulator: Simulator, assets_and_pools, allocations):
    # reset simulator for next run
    simulator.reset()

    if allocations is None:
        return sys.float_info.min

    simulator.init_data(copy.deepcopy(assets_and_pools), allocations)

    # update reserves given allocations
    # simulator.allocations = allocations
    simulator.update_reserves_with_allocs()
    # TODO: use this or just run reset()?
    updated_assets_pools = copy.deepcopy(simulator.assets_and_pools)

    initial_balance = updated_assets_pools["total_assets"]
    total_allocated = Decimal(0)
    cheating = False

    for _, allocation in allocations.items():
        total_allocated += Decimal(
            str(allocation)
        )  # This should fix precision issues with python floats

        # score response very low if miner is cheating somehow
        if total_allocated > initial_balance:
            cheating = True
            break

    # punish if miner they're cheating
    # TODO: create a more forgiving penalty system?
    if cheating:
        print(
            f"CHEATER DETECTED - PUNISHING 👊😠"
        )
        print(
            f"Allocations: {allocations} {np.sum(np.array(list(allocations.values())))}, total: {updated_assets_pools['total_assets']}"
        )
        return sys.float_info.min

    # run simulation
    simulator.run()
    # calculate aggregate yield
    pct_yield = 0
    for pools in simulator.pool_history:
        curr_yield = 0
        for uid, pool_data in pools.items():
            # print(f'uid: {uid}, pool_data: {pool_data}')
            util_rate = pool_data["borrow_amount"] / pool_data["reserve_size"]
            # util_rate = allocations[uid] / (pool_data['reserve_size'] + allocations[uid])
            pool_yield = allocations[uid] * supply_rate(
                util_rate, simulator.assets_and_pools["pools"][uid]
            )
            # print(f'uid {uid}, reserve: {pool_data["reserve_size"]} allocation: {allocations[uid]} util_rate: {util_rate} yield {pool_yield}')
            curr_yield += pool_yield
        pct_yield += curr_yield
        # print(f'curr_yield: {curr_yield} pct_yield: {pct_yield}')

    pct_yield /= initial_balance
    # print(f'pct_yield after balance: {pct_yield}, steps: {simulator.timesteps} apy: {pct_yield / simulator.timesteps * 365}')
    return(
        pct_yield / simulator.timesteps
    ) * 365  # for simplicity each timestep is a day in the simulator


def calc_strategy_apy(allocations, assets_and_pools, simulator=None, seed=None):
    simulator = Simulator(seed=seed)
    simulator.initialize()
    simulator.init_data(copy.deepcopy(assets_and_pools), allocations)

    return calculate_apy(simulator, assets_and_pools, allocations)


def strategies_apy(strategies):
    init_assets_and_pools = generate_assets_and_pools()

    result_apy = {}
    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    for k, v in strategies.items():
        allocation = strategies[k](synapse=synapse)
        result_apy[k] = calc_strategy_apy(allocation, init_assets_and_pools)
    return result_apy

def evaluate_strategies(strategies):
    """
    Similar to `strategies_apy` but using get_agg_apy
    """
    init_assets_and_pools = generate_assets_and_pools()

    result_apy = {}
    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    for k, v in strategies.items():
        allocation = strategies[k](synapse=synapse)
        result_apy[k] = get_agg_apy(allocation, init_assets_and_pools)
    return result_apy

def max_strategy():
    init_assets_and_pools = generate_assets_and_pools()

    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    equal_greedy = equal_greedy_allocation_algorithm(synapse=synapse)
    greedy_allocations = greedy_allocation_algorithm(synapse=synapse)
    sorted_greedy = sorted_greedy_allocation_algorithm(synapse=synapse)
    pick_one = pick_one_allocation_algorithm(synapse=synapse)
    print('equal')
    equal_greedy_apy = calc_strategy_apy(equal_greedy, init_assets_and_pools)
    print('original')
    original_apy = calc_strategy_apy(greedy_allocations, init_assets_and_pools)
    print('sorted')
    sorted_greedy_apy = calc_strategy_apy(sorted_greedy, init_assets_and_pools)
    print('pick_one')
    pick_one_apy = calc_strategy_apy(pick_one, init_assets_and_pools)

    max_apy = max(equal_greedy_apy, sorted_greedy_apy, original_apy, pick_one_apy)
    if max_apy != pick_one_apy:
        return greedy_allocations, equal_greedy, sorted_greedy, pick_one, synapse.assets_and_pools
    else:
        return None


def compare_strategy():
    init_assets_and_pools = generate_assets_and_pools()

    synapse = AllocateAssets(assets_and_pools=copy.deepcopy(init_assets_and_pools))
    greedy_allocations = greedy_allocation_algorithm(synapse=synapse)
    sorted_greedy = sorted_greedy_allocation_algorithm(synapse=synapse)
    equal_greedy = equal_greedy_allocation_algorithm(synapse=synapse)
    pick_one = pick_one_allocation_algorithm(synapse=synapse)

    simulator = Simulator()
    simulator.initialize()
    simulator.init_data(copy.deepcopy(init_assets_and_pools), greedy_allocations)

    equal_greedy_apy = calculate_apy(simulator, equal_greedy)
    sorted_greedy_apy = calculate_apy(simulator, sorted_greedy)
    original_apy = calculate_apy(simulator, greedy_allocations)
    pick_one_apy = calculate_apy(simulator, pick_one)

    # print(f'equal: {calculate_apy(simulator, equal_greedy)}')
    # print(f'sorted: {calculate_apy(simulator, sorted_greedy)}')
    # print(f'original: {calculate_apy(simulator, greedy_allocations)}')

    max_apy = max(equal_greedy_apy, sorted_greedy_apy, original_apy, pick_one_apy)
    # if max_apy == equal_greedy_apy:
    #     return 'equal'
    # if max_apy == sorted_greedy_apy:
    #     return 'sorted'
    # if max_apy == original_apy:
    #     return 'original'
    # if max_apy == pick_one_apy:
    #     return 'pick_one'
    if max_apy != pick_one_apy:
        return greedy_allocations, equal_greedy, sorted_greedy, pick_one, synapse.assets_and_pools
    else:
        return None


def get_agg_apy(assets_and_pools, allocations, seed=None):
    simulator = Simulator(seed=seed)
    simulator.initialize()
    simulator.init_data(copy.deepcopy(assets_and_pools), allocations)
    simulator.reset()
    simulator.init_data(copy.deepcopy(assets_and_pools), allocations)
    simulator.update_reserves_with_allocs()
    simulator.run()
    return calculate_aggregate_apy(
        allocations,
        assets_and_pools,
        simulator.timesteps,
        simulator.pool_history,
    )

# print(results.count('equal'))
# print(results.count('sorted'))
# print(results.count('original'))
# print(results.count('pick_one'))

# print(greedy_allocations)
# print(init_assets_and_pools['pools'])

# pool = init_assets_and_pools['pools']['0']
# supply_rate(pool['borrow_amount'] * 1.1 / (0 + pool['reserve_size']), pool)

# equal, sorted_greedy, greedy, pick_one = compare_strategy()

/Users/chau.hoang/miniforge3/envs/SN10/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/Users/chau.hoang/miniforge3/envs/SN10/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(


In [197]:
import itertools
from sturdy.igreedy import igreedy_allocation_algorithm, igreedy_orthogonal_allocation_allocations
from sturdy.utils.gloyiop import generate_bounds, global_yiop_allocation_algorithm
from sturdy.utils.yiop import yiop_allocation_algorithm

def calc_apy(pools, allocations):
    curr_yield = 0
    for uid, pool_data in pools.items():
        # print(f'uid: {uid}, pool_data: {pool_data}')
        util_rate = pool_data["borrow_amount"] / (pool_data["reserve_size"] + allocations[uid])
        # util_rate = allocations[uid] / (pool_data['reserve_size'] + allocations[uid])
        pool_yield = allocations[uid] * supply_rate(
            util_rate, pools[uid]
        )
        # print(f'uid {uid}, reserve: {pool_data["reserve_size"]} allocation: {allocations[uid]} util_rate: {util_rate} yield {pool_yield}')
        curr_yield += pool_yield
    return curr_yield


assets_and_pools = generate_assets_and_pools(num_pools=10)
print(assets_and_pools)
allocation = igreedy_orthogonal_allocation_allocations(AllocateAssets(assets_and_pools=assets_and_pools))
print('igreedy:', allocation)
print('igreedy:', calc_apy(assets_and_pools['pools'], allocation))
print('igreedy: calc_strategy_apy', calc_strategy_apy(allocation, assets_and_pools, seed=0))
print('igreedy: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))
print('igreedy: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))
print('igreedy: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))
print('igreedy: get_agg_apy', get_agg_apy(assets_and_pools, allocation))
allocation  = yiop_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))
print('yiop:', allocation)
print('yiop: ', calc_apy(assets_and_pools['pools'], allocation))
print('yiop: calc_strategy_apy', calc_strategy_apy(allocation, assets_and_pools))
print('yiop: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))
print('yiop: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))
print('yiop: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))
print('yiop: get_agg_apy', get_agg_apy(assets_and_pools, allocation, seed=0))

# generate_bounds(assets_and_pools['pools'])

# print(calc_strategy_apy(allocation, assets_and_pools))
# print(get_agg_apy(allocation, assets_and_pools))

# print(assets_and_pools)
# generate_bounds(assets_and_pools['pools'])

{'total_assets': 2.2, 'pools': {'0': {'pool_id': '0', 'base_rate': 0.03, 'base_slope': 0.067, 'kink_slope': 0.581, 'optimal_util_rate': 0.9, 'borrow_amount': 0.75, 'reserve_size': 1.0}, '1': {'pool_id': '1', 'base_rate': 0.02, 'base_slope': 0.02, 'kink_slope': 0.986, 'optimal_util_rate': 0.8, 'borrow_amount': 0.75, 'reserve_size': 1.0}, '2': {'pool_id': '2', 'base_rate': 0.02, 'base_slope': 0.017, 'kink_slope': 0.234, 'optimal_util_rate': 0.85, 'borrow_amount': 0.6, 'reserve_size': 1.0}, '3': {'pool_id': '3', 'base_rate': 0.04, 'base_slope': 0.069, 'kink_slope': 0.58, 'optimal_util_rate': 0.65, 'borrow_amount': 0.9, 'reserve_size': 1.0}, '4': {'pool_id': '4', 'base_rate': 0.01, 'base_slope': 0.022, 'kink_slope': 0.889, 'optimal_util_rate': 0.85, 'borrow_amount': 0.55, 'reserve_size': 1.0}, '5': {'pool_id': '5', 'base_rate': 0.04, 'base_slope': 0.061, 'kink_slope': 0.522, 'optimal_util_rate': 0.75, 'borrow_amount': 0.8, 'reserve_size': 1.0}, '6': {'pool_id': '6', 'base_rate': 0.05, 'bas

In [44]:
%load_ext autoreload
%autoreload 2
"""
Compute apy from different strategies and compare the results
"""
import pandas as pd
from sturdy.igreedy import disturb, igreedy_allocation_algorithm, igreedy_sphere_allocation_allocations, igreedy_orthogonal_allocation_allocations
from sturdy.utils.misc import greedy_allocation_algorithm
from sturdy.utils.lazy import equal_greedy_allocation_algorithm, sorted_greedy_allocation_algorithm
from sturdy.utils.lazy import equity_greedy_allocation_algorithm
from sturdy.utils.yiop import fine_yiop_allocation_algorithm, robust_yiop_allocation_algorithm, yiop2_allocation_algorithm, yiop3_allocation_algorithm, yiop_allocation_algorithm, gloyiop_allocation_algorithm
from sturdy.utils.sim_yiop import simulated_yiop_allocation_algorithm
from sturdy.utils.gloyiop import global_yiop_allocation_algorithm

strategies = {
    # 'equal': equity_greedy_allocation_algorithm,
    # 'yiop3': yiop3_allocation_algorithm,
    # 'yiop2': yiop2_allocation_algorithm,
    # 'sorted': sorted_greedy_allocation_algorithm,
    'yiop': yiop_allocation_algorithm,
    'disturb': disturb
    # 'isearch': igreedy_allocation_algorithm,
    # 'isearch_2': igreedy_orthogonal_allocation_allocations,
    # 'isearch_sphere': igreedy_sphere_allocation_allocations
    # 'robust_yiop': robust_yiop_allocation_algorithm,
    # 'global_yiop': global_yiop_allocation_algorithm,
    # 'sim_yiop': simulated_yiop_allocation_algorithm,
    # 'sim_yiop': sim_yiop_algorithm
    # 'simulated_yiop': simulated_yiop_allocation_algorithm,
    # 'fine_yiop': fine_yiop_allocation_algorithm,
    # 'gloyiop': gloyiop_allocation_algorithm,
}

results = [strategies_apy(strategies) for _ in range(1000)]
df = pd.DataFrame(results)
df['max_apy'] = df.max(axis=1)
df['best'] = df.idxmax(axis=1)

for k, v in strategies.items():
    df[k] = df[k] / df['max_apy']
df['best'].value_counts()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CHEATER DETECTED - PUNISHING 👊😠
Allocations: {'0': -0.00969816822286118, '1': 0.009658322651335852, '2': 0.21596189203902708, '3': 0.14100201481359154, '4': 0.19451319732088845, '5': 0.005547205529470263, '6': 0.059905009980023946, '7': 0.03310335761473366, '8': 0.154642740973507, '9': -0.004635572744024669} 0.7999999999556918, total: 0.8
CHEATER DETECTED - PUNISHING 👊😠
Allocations: {'0': 0.10538445323809081, '1': 0.07073459487814222, '2': 0.09684673737437749, '3': 0.08752930710747092, '4': 0.003195673388802644, '5': -0.05320596815417862, '6': 0.015052479285325543, '7': 0.017878260107003087, '8': 0.1660449357025521, '9': -0.009460472953613545} 0.4999999999739727, total: 0.5
CHEATER DETECTED - PUNISHING 👊😠
Allocations: {'0': 0.37080721346328765, '1': 0.25373230554529574, '2': 0.17690264502989242, '3': 0.10064218792987233, '4': 0.016810772022354845, '5': 0.42051070649949657, '6': 0.13987326150419804, 

In [27]:
import numpy as np
data = np.load('sphere_points.npy')
len(data)

87

In [154]:
import math
arr = np.random.rand(10)
# arr = np.array([0.19672040316784697, 0.0949434832017978, 0.030252320998443945, 0.10094905740872444, 0.010187690171803944, 0.012010955776671463, 0.20076079767696084, 0.0, 0.0, 0.0517036752023535])
count = len(arr)
diff = math.sqrt(0.1000000000001 ** 2 / count)
arr2 = arr + diff
print(diff)

# arr2 = arr.copy()
# # arr2[0] += 0.1
# arr2 += 0.1
np.linalg.norm(arr - arr2)
# np.linalg.norm([math.sqrt(0.005), math.sqrt(0.005)])

0.03162277660171541


0.10000000000010002

In [180]:
greedy_allocations, equal_greedy, sorted_greedy, pick_one, assets_and_pools = [item for item in results if item != None][0]
simulator = Simulator()
simulator.initialize()
print(f'assets and pools: {assets_and_pools}')
print(pick_one)
print('==equal', equal_greedy)
equal_greedy_apy = calculate_apy(simulator, assets_and_pools, equal_greedy)
print('==sorted')
sorted_greedy_apy = calculate_apy(simulator, assets_and_pools, sorted_greedy)
print('==original')
original_apy = calculate_apy(simulator, assets_and_pools, greedy_allocations)
print('==pick one')
pick_one_apy = calculate_apy(simulator, assets_and_pools, pick_one)

print('==custom')
arr = [2.47262559e-01, 5.11797341e-02, 1.02159377e-01, 1.11038600e-01,
 7.08541043e-02, 0.00000000e+00, 7.54874975e-02, 5.14403109e-18,
 5.88625333e-02, 2.83155594e-01]
allocations = {str(i): arr[i] for i in range(len(arr))}
custom_apy = calculate_apy(simulator, assets_and_pools, allocations)
# custom_apy = calculate_apy(simulator, assets_and_pools, {'0': 0.67, '1': 0.33})

print(assets_and_pools['pools'])
print(f'==> equal: {equal_greedy_apy}, sorted: {sorted_greedy_apy}, original: {original_apy}, pick_one: {pick_one_apy}, custom: {custom_apy}')
print(greedy_allocations)
print(pick_one)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 greedy_allocations, equal_greedy, sorted_greedy, pick_one, assets_and_pools = [item for     │
│    2 simulator = Simulator()                                                                     │
│    3 simulator.initialize()                                                                      │
│    4 print(f'assets and pools: {assets_and_pools}')                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: not enough values to unpack (expected 5, got 2)

In [ ]:
from sturdy.validator.reward import sigmoid_scale

# sigmoid_scale(3, 10)

# => 9945
# => 992
miner_1 = 16.505147166086775
miner_2 = 16.288483029869116

miner_apy = miner_2
max_apy = 16.505147166086775
axon_time = 1
num_pools = 10

(0.2 * sigmoid_scale(axon_time, num_pools=num_pools)) + (
    0.8 * miner_apy / max_apy
)

0.9888088579076895